# Prism V2 Demo - Local Agents

# Imports

In [ ]:
%autoawait
import time
import datetime
import re
import qrcode
import uuid
import base64
import jwt
# https://pypi.org/project/termcolor/
from termcolor import colored,cprint
import json
import os
# from jwcrypto import jwk, jws,jwt
from jwcrypto.common import json_encode

from dotenv import load_dotenv
load_dotenv()
APIKEY = os.getenv('APIKEY')

from peerdid.peer_did import (
    resolve_peer_did,
)

from IPython import display
from base64 import b64decode

# display.Image(b64decode(base64_data)

## Instantiate K8 Authenticated Agent Controllers

In [ ]:
# from prism_agent_open_api_specification_client import AuthenticatedClient

# government_client_controller = AuthenticatedClient(base_url="https://agent-df56h.atalaprism.io/prism-agent", token=APIKEY)
# citizen_client_controller = AuthenticatedClient(base_url="https://agent-kj46b.atalaprism.io/prism-agent", token=APIKEY)
# university_store_client_controller = AuthenticatedClient(base_url="https://agent-sd98k.atalaprism.io/prism-agent", token=APIKEY)
# dapp_store_client_controller = AuthenticatedClient(base_url="https://agent-sd98k.atalaprism.io/prism-agent", token=APIKEY)
# employer_store_client_controller = AuthenticatedClient(base_url="https://agent-sd98k.atalaprism.io/prism-agent", token=APIKEY)

## Instantiate Localhost Agent Controllers

In [ ]:
from prism_agent_open_api_specification_client import Client

government_client_controller = Client(base_url="http://host.docker.internal:8080/prism-agent")
citizen_client_controller = Client(base_url="http://host.docker.internal:8090/prism-agent")
university_client_controller = Client(base_url="http://host.docker.internal:9000/prism-agent")
employer_client_controller = Client(base_url="http://host.docker.internal:9010/prism-agent")
dapp_store_client_controller = Client(base_url="http://host.docker.internal:9030/prism-agent")

# Connections

## Import Connection Management Models and API Client

In [ ]:
from prism_agent_open_api_specification_client.models import ConnectionCollection,Connection,ConnectionInvitation,CreateConnectionRequest,AcceptConnectionInvitationRequest
from prism_agent_open_api_specification_client.api.connections_management import get_connections,get_connection,create_connection,accept_connection_invitation
from prism_agent_open_api_specification_client.types import Response

## Connections Helper Functions

In [ ]:
def create_invitation(inviter_client, label):
    # Create Invitation from Requestor
    conn_request = CreateConnectionRequest()
    conn_request.label = label
    invitation: Response[Connection] =  create_connection.sync(client=inviter_client,json_body=conn_request)
    # print("Connection ID\n",invitation.connection_id)
    # print("Invitation URL\n",issuer_invitation.invitation.invitation_url)
    invitationUrlbase64 = re.sub("^.*_oob=", "", invitation.invitation.invitation_url)
    # print("Invitation Base64\n",issuer_invitationUrlbase64)
    return invitation, invitationUrlbase64


def accept_connection(invitee_client, invitationUrlbase64):
    # Accept Invitation
    accept_conn_request = AcceptConnectionInvitationRequest(invitationUrlbase64)
    invitation: Response[ConnectionInvitation] =  accept_connection_invitation.sync(client=invitee_client,json_body=accept_conn_request)
    return invitation

async def check_connection_state(inviter_client, inviter_invitation, invitee_client, invitee_invitation):
    # Check Connection State
    inviter_conn: Response[Connection] =  get_connection.sync(client=inviter_client,connection_id=inviter_invitation.connection_id)
    invitee_conn: Response[Connection] =  get_connection.sync(client=invitee_client,connection_id=invitee_invitation.connection_id)
    # print(inviter_invitation.connection_id,invitee_invitation.connection_id)
    # print(inviter_conn.connection_id,invitee_conn.connection_id)

    while (inviter_conn.state != 'ConnectionResponseSent' and invitee_conn.state != 'ConnectionResponseReceived'):
        inviter_conn: Response[Connection] =  get_connection.sync(client=inviter_client,connection_id=inviter_invitation.connection_id)
        invitee_conn: Response[Connection] =  get_connection.sync(client=invitee_client,connection_id=invitee_invitation.connection_id)
        print("Inviter State: {} Invitee State: {} \n".format(inviter_conn.state,invitee_conn.state))
        time.sleep(3)

    print("Connection established between Inviter and Invitee!")
    
    return inviter_conn.state,invitee_conn.state

async def auto_connection(inviter_client, invitee_client, label):
    invitation,invitationUrlbase64 = create_invitation(inviter_client, label)
    # print("\n\n\n",invitation.connection_id)
    accept_invitation = accept_connection(invitee_client, invitationUrlbase64)
    await check_connection_state(inviter_client, invitation, invitee_client, accept_invitation)
    # print("\n\n\n", accept_invitation.connection_id)
    inviter_conn: Response[Connection] =  get_connection.sync(client=inviter_client,connection_id=invitation.connection_id)
    invitee_conn: Response[Connection] =  get_connection.sync(client=invitee_client,connection_id=accept_invitation.connection_id)
    return inviter_conn, invitee_conn


# Credentials

## Import Credential Management Models and API Client

In [ ]:
from prism_agent_open_api_specification_client.models import IssueCredentialRecord, CreateIssueCredentialRecordRequest, CreateIssueCredentialRecordRequestClaims, IssueCredentialRecordCollection
from prism_agent_open_api_specification_client.api.issue_credentials_protocol import get_credential_record, get_credential_records, create_credential_offer,accept_credential_offer,issue_credential
from prism_agent_open_api_specification_client.types import Response, Unset

## Import DID Management Models and API Client

In [ ]:
from prism_agent_open_api_specification_client.models import DID, DIDDocumentMetadata, DIDOperationResponse, DIDResponse, CreateManagedDidRequest, ListManagedDIDResponseInner, CreateManagedDIDResponse
from prism_agent_open_api_specification_client.api.did import get_did
from prism_agent_open_api_specification_client.api.did_registrar import create_managed_did,list_managed_did,publish_managed_did
from prism_agent_open_api_specification_client.types import Response

## Credential Helper Functions

In [ ]:
def create_prism_did(did_client):
    data = {
    "documentTemplate": {
      "publicKeys": [
        {
          "id": "auth0",
          "purpose": "authentication"
        }
      ],
      "services": []
    }
  }
    new_did = CreateManagedDidRequest.from_dict(data)

    did: Response[CreateManagedDIDResponse] = create_managed_did.sync(client=did_client, json_body=new_did)
    return did

def get_credential_requests():
    # TODO - fix return 
    government_creds: Response[Credential] = get_credential_records.sync(client=government_client_controller)
    citizen_creds: Response[Credential] = get_credential_records.sync(client=citizen_client_controller)
    university_creds: Response[Credential] = get_credential_records.sync(client=university_client_controller)
    employer_creds: Response[Credential] = get_credential_records.sync(client=employer_client_controller)
    dapp_store_creds: Response[Credential] = get_credential_records.sync(client=dapp_store_client_controller)
    # print("Issuer Proofs:\n{}\n\nHolder Proofs:\n{}\n\nVerifier Proofs:\n{}".format(issuer_proofs, holder_proofs,verifier_proofs))
    return government_creds,citizen_creds,university_creds, employer_creds, dapp_store_creds

def create_credential(schema, schemaId, subjectDid):
    cred_claims = CreateIssueCredentialRecordRequestClaims()
    cred_claims = cred_claims.from_dict(schema)
    # print(cred_claims)
    credential = CreateIssueCredentialRecordRequest(subject_id=subjectDid, claims=cred_claims, schema_id=schemaId, validity_period=3600, automatic_issuance=False, await_confirmation=False)
    # print(credential_offer.to_dict())
    return credential

def send_credential_offer(issuer_client,credential):
    # Create Credential Offer
    cred_offer: Response[IssueCredentialRecord] = create_credential_offer.sync(client=issuer_client,json_body=credential)
    # print(cred_offer)
    return cred_offer

async def accept_credential(citizen_client, cred_record_id):
    # citizen_creds: Response[IssueCredentialRecord] = get_credential_record.sync(client=citizen_client,record_id=cred_record_id)
    # # print(citizen_creds)
    # # Check Credential State
    # while (citizen_creds.protocol_state != 'OfferReceived'):
    #     try:
    #             citizen_creds: Response[IssueCredentialRecord] = get_credential_record.sync(client=citizen_client,record_id=cred_record_id)
    #             time.sleep(1)
    #             # print(citizen_creds.protocol_state)
    #     except IndexError as e:
    #         pass

    # print(colored("Current state for Issuer Credential Record {} is {} and Holder Credential Record {} is {}".format(issuer_creds.items[0].record_id,issuer_creds.items[0].protocol_state,holder_creds.items[0].record_id,holder_creds.items[0].protocol_state), "magenta", attrs=["bold"]))
    # print(holder_creds.items[0].record_id)
    citizen_cred_record: Response[IssueCredentialRecord] = accept_credential_offer.sync(client=citizen_client, record_id=cred_record_id)
    # print(citizen_cred_record)
    return citizen_cred_record

def issue_credential_record(issuer_client, record_id):
    metro_cred_record: Response[IssueCredentialRecord] = issue_credential.sync(client=issuer_client, record_id=record_id)
    # print(metro_cred_record)
    return metro_cred_record

def check_credential_state(issuer_client, issuer_record_id, holder_client, holder_record_id):
    issuer_creds: Response[IssueCredentialRecord] = get_credential_record.sync(client=issuer_client, record_id=issuer_record_id)
    holder_creds: Response[IssueCredentialRecord] = get_credential_record.sync(client=holder_client, record_id=holder_record_id)

    print(colored("Current state for Issuer Credential Record {} is {} and Holder Credential Record {} is {}".format(issuer_creds.record_id,issuer_creds.protocol_state,holder_creds.record_id,holder_creds.protocol_state), "magenta", attrs=["bold"]))

    while (issuer_creds.protocol_state != 'CredentialSent' and holder_creds.protocol_state != 'CredentialReceived'):
        issuer_creds: Response[IssueCredentialRecord] = get_credential_record.sync(client=issuer_client, record_id=issuer_record_id)
        holder_creds: Response[IssueCredentialRecord] = get_credential_record.sync(client=holder_client, record_id=holder_record_id)
        print("Issuer State: {} Holder State: {} \n".format(issuer_creds.protocol_state,holder_creds.protocol_state))
        time.sleep(2)

    print(colored("Credential Record {} Issued to Holder! Continue with notebook".format(issuer_creds.record_id), "green", attrs=["bold"]))
    return issuer_creds, holder_creds



# Presentations

## Import Presentation Management Models and API Client

In [ ]:
from prism_agent_open_api_specification_client.models import PresentationStatus, ErrorResponse, Proof, ProofRequestAux, PublicKeyJwk, RequestPresentationInput, RequestPresentationOutput, RequestPresentationAction
from prism_agent_open_api_specification_client.api.present_proof import get_all_presentation, request_presentation, update_presentation, get_presentation
from prism_agent_open_api_specification_client.types import Response

## Presentation Helper Functions

In [ ]:
def get_all_presentation_requests():
    government_proofs: Response[PresentationStatus] = get_all_presentation.sync(client=government_client_controller)
    citizen_proofs: Response[PresentationStatus] = get_all_presentation.sync(client=citizen_client_controller)
    university_proofs: Response[PresentationStatus] = get_all_presentation.sync(client=university_client_controller)
    employer_proofs: Response[PresentationStatus] = get_all_presentation.sync(client=employer_client_controller)
    dapp_store_proofs: Response[PresentationStatus] = get_all_presentation.sync(client=dapp_store_client_controller)
    # print("Issuer Proofs:\n{}\n\nHolder Proofs:\n{}\n\nVerifier Proofs:\n{}".format(issuer_proofs, holder_proofs,verifier_proofs))
    return government_proofs,citizen_proofs,university_proofs, employer_proofs, dapp_store_proofs

def get_client_presentation_requests(client):
    client_proofs: Response[PresentationStatus] = get_all_presentation.sync(client=client)
    return client_proofs

def initiate_proof_request(verifier_client, verifier_connection_id):
    data = {
        "connectionId": verifier_connection_id, 
        "proofs": []
    }
    # print(data)

    # data = {
    #   "connectionId": verifier_invitation.connection_id, 
    #     "proofs": []
    # }
    proof_request = RequestPresentationInput.from_dict(data)
    # print(proof_request)

    verifier_proof_request: Response[RequestPresentationInput] = request_presentation.sync(client=verifier_client, json_body=proof_request)

    print(verifier_proof_request)
    return verifier_proof_request

def accept_proof(holder_client, holder_cred_id, holder_proof_id):
    from typing import Any, Dict, Optional, Union, cast

    data = {
      "action": "request-accept",
      # "proofId": [holder_creds.items[0].record_id]
        "proofId": [holder_cred_id]
    }

    proof_accept = RequestPresentationAction.from_dict(data)

    holder_proof_request: Response[Union[Any, ErrorResponse]] = update_presentation.sync(client=holder_client, json_body=proof_accept, id=holder_proof_id) #holder_proofs[0].presentation_id)

    print(holder_proof_request)
    return holder_proof_request

def check_presentation_status(holder_client,  verifier_client):
    # TODO - fix presentation record check to specific ID instead of all records
    holder_proofs: Response[PresentationStatus] = get_all_presentation.sync(client=holder_client)
    verifier_proofs: Response[PresentationStatus] = get_all_presentation.sync(client=verifier_client)
    # Check Proof Presentation State
    while (holder_proofs[0].status != 'PresentationSent' and verifier_proofs[0].status != 'PresentationVerified'):
        holder_proofs: Response[PresentationStatus] = get_all_presentation.sync(client=holder_client)
        verifier_proofs: Response[PresentationStatus] = get_all_presentation.sync(client=verifier_client)
        print("Verifier State: {} Holder State: {} \n".format(verifier_proofs[0].status,holder_proofs[0].status))
        time.sleep(1)

    print("Proof Presentation completed between Verifier and Holder!")
    
def check_presentation_record_status(holder_client, holder_record_id,  verifier_client, verifier_record_id):
    # TODO - fix presentation record check to specific ID instead of all records
    holder_proof: Response[PresentationStatus] = get_presentation.sync(client=holder_client, id=holder_record_id)
    verifier_proof: Response[PresentationStatus] = get_presentation.sync(client=verifier_client, id=verifier_record_id)
    # Check Proof Presentation State
    while (holder_proof.status != 'PresentationVerified' and verifier_proof.status != 'PresentationSent'):
        holder_proof: Response[PresentationStatus] = get_presentation.sync(client=holder_client, id=holder_record_id)
        verifier_proof: Response[PresentationStatus] = get_presentation.sync(client=verifier_client, id=verifier_record_id)
        print("Verifier State: {} Holder State: {} \n".format(verifier_proof.status, holder_proof.status))
        time.sleep(1)

    print("Proof Presentation completed between Verifier and Holder!")
                                                                                         

# DIDs

## Import DID Management Models and API Client

In [ ]:
from prism_agent_open_api_specification_client.models import DID, DIDDocumentMetadata, DIDOperationResponse, DIDResponse, CreateManagedDidRequest, ListManagedDIDResponseInner, CreateManagedDIDResponse
from prism_agent_open_api_specification_client.api.did import get_did
from prism_agent_open_api_specification_client.api.did_registrar import create_managed_did,list_managed_did,publish_managed_did
from prism_agent_open_api_specification_client.types import Response

# Demo Use Case

# Government Flow

## Make a Connection between Metropolgov.io and Citizen

In [ ]:
metropolgov_conn, citizen_conn = await auto_connection(government_client_controller, citizen_client_controller, "Connection with Metropolgov.io")

## Metropolgov.io Issue Natural Person Credential to Citizen

In [ ]:
# citizen_prism_did = create_prism_did(citizen_client_controller)
# print(citizen_prism_did)
citizen_id_schema = {
    "firstname": 'Wilmien',
    "lastname": 'Du Preez',
    "birthdate": '11/11/1999',
    "placeofbirth": "Pretoria",
    "gender": "Female",
    "idnumber": "19991111538708",
    "issuedate": "21/12/2022",
    "expiry_date": "23/12/2022",
    "photo": "data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAQ4AAAC6CAMAAABoQ1NAAAAAjVBMVEX+/v4AAADt7e3////s7Oz9/f36+vr39/fv7+/y8vLp6ena2trl5eXV1dV4eHi3t7fKysrg4OCqqqpERERlZWXGxsaWlpaenp5UVFS/v7+AgIBsbGxOTk7Ozs6kpKSJiYkbGxs0NDQmJiYRERGxsbEiIiJoaGhTU1NeXl4vLy9JSUkYGBg8PDyNjY10dHQyCsxLAAAPMElEQVR4nO1da1/qPAzf1m1tuQoqIhxUvB31qN//4z29DNjWy9Jule08J28kPyGE/9o0SdM0QhEjhBNORDJUMFQyRDBYMumJQZlg0lwwuWQydBKWDlAyiSIpOrWKTsuiUyw+cxAtmINowUhhB6WHJDnxggNZRaMWSp9Z8j84/sFhgwMJoiknUmaoZEiZEa+ZaE6ZZHLB5JLJBIMlM0DJSYQpJyyJOjC4iXER1hfJUVKAF4l1qAAvEutQAZ4cS/KxyIFZPJYSI0bp6bEIJh2c5AhFSS8mbU8kR//g+AeHIxwVH1YVTSuiOZNWRFOz0jXJ3L4TQQh1K9lT50hqgzNOhWq5ymDJZCWGSoaqjBSW2SVTvg6yDyymY0YLLFb8TiS30flsfgfe3XxfvsRHery8ul9O8zN7NHLG/OykxShZvsVaelrfjFHxnf8PJ53S+ZUei8M42VxgSv4fcBA631vBEPR+l9LkfwAHnV42gyHofiQWm786hMP3QDA43Z3DlIoAhuBcEFEZ+RpXGBkbSYaqjPx8pgjDkw8HNOJ4f42BkrvSOfd2w1KtS1NmqqM0Tei1ExicZiPSLLlLnX/OScdzZzSYTd3l53fSQ8BBnz3QYPT8V8JBb/3QiOMVX2HOAYdtG6Kl7UA7XzTieMPwMEruWOeoYljLTMUWNzGaz1cY+uAPB8PDIrlbnX/E70jougUabIGhf1UIR1et0IjjG/oXOelk2hKNOJ4jreQhwkEoNE6x0EK1jgOFgy7boxG/89WiNyEc9jelSdoBGmK5dTSlPjpHRS2EwCsTrw+JBskU+/2SKR5LYn0sh8KKpGDa2tGCbmldcvc6d+CVVlwadZTmuBs04nhEOvNKTTqHd9LRV1dwfOPhxyyEdIVGHO/I4OGgNw6/d76x/vuD/gAcfn430DtOyDscjRlCC8OGQwFX3ipWAOgc4aCUX8DReGBPCKHbP+Z37POw2mIcNIRjwmdwOC6KYf1tGR7Ub5DCq38i98UCPmkRfgKjMStmLxsgxvdcUt8FrhdOuuWX1ekhL+DgX/Npehf3xQYMh9UyVmh3RIMPENO25RqfAQ7VabNk9C1wIApGY1NCg+Nh2p+aErCj6aEzg0OUNRS1EJIpyh8kU9RSSCY5MSxY4K9ZsMCZXDJZSVgSZfC58oIqcDA8DJHOFxaSA+lMonBrOHuxhcIxraLBAdHj8RiF9TsSR3PgMGlJ+qL9SSrdK2iwz+tdVLYcD9RJJ9DdBJHbUeDQ2+HZYOEAZ4zVqSK/51Hz3ic2LQYKB3A74UuLBpOgTTlfBIUjpCmFobFHBjj05nQT1JQ6lT84bX9lwP2EsQkNZj726tsfsW5jrSOdo7IX47LtCtgche3Zr4xoMFG6kpAdCaZzUCcdFM1+GqeKGB6aeqEVHWbMohnpKi0saDBZmnzJepBwJKAUum2qiOHxS/1MEhAO33moL8oqzUMytqBw2Ka0ThXxlZoSql0wnZNwKwu2WNJfh/9NGtBgSqufvkPhVpZwa7glLb4onvlNExoMD9Ugz+gQQzjNrC/ou4j8L5umChejBj6POJTOIZ108+CgxYLROFW4GI2cCRkcHPVM2Ot8MRpdbOQUkY/8DoAGE6RGPtcDhKOysGxHqAhk1vE7+8P/+RswVbggdR/vjgaDI5gpLScGN8dfjqKPaxShBXCqRNq4dkODmdKGo2KVs3IVJtMcQisJw+VotDQtEF/0EQt2lzA02ONT4HhDYXQmpOyGdXpEkZY308oZjcKwrGFThdtSZevqQRQiBzhWGc5Jx2ULuKn9dOZcjYBosDerlXZcg2HFLLS8df9dhwM/g9HQOWLjocFRLXO5Un48HI1IswX1HAwOJ8MLX4bIoqz+L4dfr8Khpgi/gugs+ncEosry2Bi4WuFQ61LXKJDWwap/KnmbR91GChgOdWvzkSbDCuGqj/QP7hYOHrUMykmvTXjSMRzjocFRLWkCOuRgOK6HBkc1BW7eSwHAockPPtPAcEizJEUXZkmKLsySFF1ijmYpEQ6xNEsnYdX91Ys2cGiyjCsaQmdR3Heywk7V/5HGpEcHYcmkqv68DRyaSt37ADoLJnIYS3APr37ACRq+auHQFEYxr39ITjqtzfeG7RQ7HJrdvPXA4Kg51vWQ1gkOTdnLflhw4FqCc9sGDk2V6UvJOnYKh5MPCw2HqgEcow9/ODTpH1kDFK5/B7SdYa4wxxPLgjl8RFkM3v1jOH1x6gR3rbNgumzBGB0fS6aUDFJ/OLR7vWPStc6CCeCVJlhz4AueC1Tg0NbqTklYr7RDOHSHWHb+cGjPS+0GAwfV1YT5u6VIW7r8k3AU89Ak2lJYwaeKtsUPbANSC4e2dPmiDoe/zmUmOnQgzPP81MFQYQ7tDPMTQyVDqwzF+tpabz9MV+DB4aDd6XwEIO+4BSNCpjZpr95w6IuIdn3v38HgJhfGOmPv5LEmccxpGsgrbQ2HzBFQTG43lUOONV/SN1tqKMcc9xEOKpuwkvHzrKlHh2960FCOueghHOn9+u3tEtbl6NYPDmQoxwwFRxtTmpAluOOTZ8bD1GFrFCqEc1pos8qilWUoW+pOlT+qQeibJxyGMzEL+EJb0TlrWGjbumGYzOvLycul5uD4g99GHPqtReNpFMgN08EhFAE7vJSOV6VthM+tvoLSK4gzOGHxS+iNhRaiCaWYXtxttm9Xm7ubreGcue74XzMchkruzx7DkU4nwlJlo/n9Xq8+J0CJsYLGSHccjtFVMDgKwxt5p1ISwsfDn+bmA867C/rDcJw2Bzi6Tv90kBzMoV0p3PBAluNSd3nnCU2ZHOwihAN37XWoCLOcw2d03V3Y2XkI59ClA44Hym3dHUZ93mexndWoETQrZjSigv5EvYZjYlG9RpbxUcoBMN98bcGYl971Fw6X4WG2p4gc6vgRuon3S/N5GFHFfM4QrqmdIbxvidn/QHQW739vt9urX03dtXZd6KwtaEgK8OS7JXjFuyV4xbulS1NiKqUj2KVD670hM8aWKKBNHpEOdNaWuxzggIwlm4fn0vj7zZQpRIZUYI0ucTc6KzOru30WNAb3j9uzkMOIBwW0QFmFO97TFRxs6VvuYXD8pg989mvRuGCGYbR8ta2ysh9S3+HgJmn6vHldX/5q6NvxC+Wvsjl+HQw8i7diSkd0sTPeyfAZ8qSkiw/btAwVZ3OThtGBuI34WFQBYdySp1SlkrbV6i7gOdqmonUPyhrsIc8TosVlPJug0xk5hJ9f4vvS9oMp8xPHabhOjJ1X/zCB2d4Oh9igRGj+J/59W7TcRNNvfhtLebygzPDxLe1aZ3Q6wBE5TK0mD08qEFlaSgpaHebD9W82VG7mt8+bp/h9SeuTx7DGHNqZ9NdJl6LT1d3y+aY5+XEI4/hzXIrugperMVJMq65EjtEbHUgLRuiNNKUOJnLUykFVJ83xajk4graN14iu5N2TEqMTLQyBFI3vQHDUthiMLqq2LeWWdtCCsaRzDY6mGyiNR3R1d1vaElinHwRNeehs0NOEFl0Xu9K5eqzYxfA2r+EUgAd0t1ZzfpbbHcfF4ryHvwzFP+XnCyxtQPVSXU5X4ny1izk4c6kcvra0suYE3bvWVQy+y82EAcGR0JF9qYUWeui6w4zdF4tzw5GQbGzJ68XXwLmiKey4zYLDEcAsMWFT864AcG9ScwLu4zZ197t70YIRZztTM2vgOqv3Oj5WO4plLUa/WjDaXJqE0nS3MhQGfYLgMAZw8dPV3ah4xl3q3L2TLkWzETeZf1tSfKCF1n7N4Of9BeVXxnelcyg42EMbPTf0igedqW32b9erixQzZ7SvcPCr7cc3zZeegfYlTdF9hR62yzEWIc/ZWzDWW6ESnN7O7BnfgiBlQFqXVEufXzzJ2Fn71m5Wljy7Bl+nAKkh1B9iMQkco361YKSLV7j2T4BON9rwzUwzkvUnhCOO9wI21xCihly8Qnspsw9OOiHgqxQkNfcn0DVlaCCRYusBHGRki1B01OymI4e5V8ajB3CMnO8TrXcRU9HwuWnwgSl0/hAOu44NgJsOr70r02XULuwUptT36pdMXv2CHep+jtRUnu5STFSiGYbprL2uRr6t+H7fdoZ+t9s3uOkIOdytV6ZnDNE5nJNOHGrkStTgpsNd0ho9kfPGLBh+s1eZGtx0J5e0QpuzwkG9TB6vaLDD4X/3tbzy+jyHvxLvO7utcKCRNxq81bhdZ/vS2aoFY+45OOI4teEBvMhET7ftaj9aVf/4WY64wU03XhkIoT1vznieEM5vWeFka1BgrvoBEe83dx4n3f+Wapub7ueSHumdkvPAoXa0AdPeBofDFbY6eqZngYO61BnXyeymowh8UllPlwA4VLf7AEfRrke8zsTrpIBDMkJ0hCVTiJYfaSoBs5Hlti/bLTcguiZGnW39ioqvL1lhp+p/nyD8SGY33XSsGk6vbicWujn85VSTr5DposDIVCDnQhNq0Dmgk67vUASltQmONi7pgb7wj8OBcvil3Rp6MNbHuWWhtfQ0Ij8OB/RyVQOZ3HRD9aQble9v+aEQDlIjaCFDW7WWLmlBnyTxC+GOLRiNm1SGHSvtXbEOZHDT2xnoI+0ywy5bmdHt0pXdMMCtmXIypQnR9NFzopkBjjbOzIm2CKk6QzahPb1SSP2olfTZdN01X16U1uEI66TTlnPFUPTS1kAfafWjcLSeK4abRlsEyVV6ilB3cFRshwoH7QIOrZsOu7MVQtfIz3YIu0oKw0oEk6sMlsyxZA+3XgF0bnqLhFKNrnCm6AwpYPH2O9rOcl3RC4IdAGmmL0p+uH+HuRENjB41RS+WWzmdaI7Jj++zdO+mOxe56Olh8vMhHBNNJ/oWZ0DaIYU6cUnf8Jl24Uj96gAnEncOY9GjiG+kp+lo5FHkotASoTPtwjFvbNwyz9sxva9wk84NprTpqFhePiqmHJyL0GjMaTESNG5gJuL1hL+cTieCFlNBBSPfJj+ycJecY9qsc+3gHC0B4N+C8VQbJkQX40qKLsaVhLPMyCdRHN2T/8jLDFaFOUmG6qxm0sN10feZtD2RHPIu639w/AVwFIY3irpuZ5gOUHJUJMokUQcGNzEuwvoiOdQFrKljYUU/JIe7rbgv5qAXV139g+PvhkNX/nAQHVlFRw1K91jysZ1JYkU6KYtOypsySdmjT4oVTrx2KqzoiWTyH0OmNgTTsTIxAAAAAElFTkSuQmCC",
  }

metropolgov_cred = create_credential(citizen_id_schema, "schema:12345:gov", metropolgov_conn.their_did)
metropolgov_cred_offer = send_credential_offer(government_client_controller, metropolgov_cred)
metropolgov_record_id = metropolgov_cred_offer.record_id
print(metropolgov_record_id)
time.sleep(2)

In [ ]:
citizen_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=citizen_client_controller)
citizen_gov_record_id = citizen_creds.items[0].record_id
print(citizen_gov_record_id, ' ', citizen_creds.items[0].protocol_state)

In [ ]:
citizen_cred = await accept_credential(citizen_client_controller, citizen_gov_record_id)
print(citizen_cred)
time.sleep(2)

In [ ]:
time.sleep(3)
metro_cred_record = issue_credential_record(government_client_controller, metropolgov_cred_offer.record_id)

metropolgov_credentials, citizen_credential = check_credential_state(government_client_controller, metropolgov_record_id, citizen_client_controller, citizen_gov_record_id)


## JWT Verify

### List DIDs in Wallet

In [ ]:
governance_dids: Response[ListManagedDIDResponseInner] = list_managed_did.sync(client=government_client_controller)
print(governance_dids)

### Resolve DID

In [ ]:
governance_resolved_did: Response[DIDResponse] = get_did.sync(client=government_client_controller, did_ref=governance_dids[0].long_form_did)
print(governance_resolved_did.to_dict())

### Decode JWT in Credential

In [ ]:
token = citizen_credential.jwt_credential
# print(token)
jsondata = base64.urlsafe_b64decode(token)
# print(jsondata)
b64decoded_token = json.loads(jsondata.decode())
jwk = governance_resolved_did.did.authentication[0].public_key_jwk
# print(jwk)
# print(json.dumps(jwk.to_dict()))
pubKey = jwt.algorithms.ECAlgorithm.from_jwk(json.dumps(jwk.to_dict()))
jwt.decode(b64decoded_token , key = pubKey, algorithms=["ES256K"])

In [ ]:
b64decoded_token = b64decoded_token + 'a'
jwt.decode(b64decoded_token , key = pubKey, algorithms=["ES256K"])

# University Flow

## Make a Connection between University and Citizen

In [ ]:
uni_conn, citizen_uni_conn = await auto_connection(university_client_controller, citizen_client_controller, "Connection with University of I&T")

## Citizen Present Natural Person Proof to University of Innovation and Technology

In [ ]:
uni_proof_request = initiate_proof_request(university_client_controller, uni_conn.connection_id)
print(uni_proof_request.presentation_id)

In [ ]:
government_proofs,citizen_proofs,university_proofs, employer_proofs, dapp_store_proofs = get_all_presentation_requests()
print(government_proofs,citizen_proofs,university_proofs, employer_proofs, dapp_store_proofs)

In [ ]:
print(citizen_proofs[0].presentation_id)
print(university_proofs[0].presentation_id)

In [ ]:
citizen_uni_proof_request = accept_proof(citizen_client_controller, citizen_gov_record_id, citizen_proofs[0].presentation_id)

In [ ]:
check_presentation_record_status(university_client_controller, university_proofs[0].presentation_id, citizen_client_controller, citizen_proofs[0].presentation_id)

## JWT Verify

### Decode JWT in Credential

In [ ]:
citizen_proofs = get_client_presentation_requests(citizen_client_controller)

In [ ]:
# print(citizen_proofs)

In [ ]:
# print(citizen_proofs[0].data[0])

In [ ]:
token = citizen_proofs[0].data[0]

try:
    jwt_gov_decoded_presentation = jwt.decode(token, options={"verify_signature": False})
    print(json.dumps(jwt_gov_decoded_presentation, indent=2))
except (jwt.ExpiredSignatureError, jwt.InvalidAudienceError) as e:
    print("[ERROR]", e)    

In [ ]:
token = jwt_gov_decoded_presentation['vp']['verifiableCredential'][0]

try:
    jwt_gov_decoded_vc = jwt.decode(token, options={"verify_signature": False})
    print(json.dumps(jwt_gov_decoded_vc, indent=2))
except (jwt.ExpiredSignatureError, jwt.InvalidAudienceError) as e:
    print("[ERROR]", e)
    


In [ ]:
jwt_nbf = jwt_decoded_id_token['nbf']
jwt_exp = jwt_decoded_id_token['exp']
print('NBF epoch: {} EXP epoch: {}'.format(jwt_nbf,jwt_exp))

human_jwt_nbf = datetime.datetime.fromtimestamp(jwt_nbf)
human_jwt_exp = datetime.datetime.fromtimestamp(jwt_exp)
print('NBF: {} EXP: {}'.format(human_jwt_nbf,human_jwt_exp))

### Resolve DID

In [ ]:
resolved_did: Response[DIDResponse] = get_did.sync(client=government_client_controller, did_ref=jwt_gov_decoded_vc['iss'])
print(resolved_did.to_dict())

In [ ]:
token = jwt_gov_decoded_presentation['vp']['verifiableCredential'][0]

jwk = resolved_did.did.authentication[0].public_key_jwk
pubKey = jwt.algorithms.ECAlgorithm.from_jwk(json.dumps(jwk.to_dict()))
jwt.decode(token , key = pubKey, algorithms=["ES256K"])

In [ ]:
token = token + 'x'
jwt.decode(token , key = pubKey, algorithms=["ES256K"])

## University of T&I Issue Education Credential to Citizen

In [ ]:
# citizen_prism_did = create_prism_did(citizen_client_controller)
# print(citizen_prism_did)
citizen_univ_schema = {
    "firstname": 'Wilmien',
    "lastname": 'Du Preez',
    "course": 'Web3 Developer',
    "grade": "First Class Honours",
    "completion_date": "01/12/2022",
    "issuedate": "21/12/2022",
    "expiry_date": "23/12/2022"
  }

univ_cred = create_credential(citizen_univ_schema, "schema:12345:univ", uni_conn.their_did)
uni_cred_offer = send_credential_offer(university_client_controller, univ_cred)
time.sleep(2)

In [ ]:
citizen_uni_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=citizen_client_controller)
citizen_uni_record_id = citizen_uni_creds.items[1].record_id
print(citizen_uni_creds.items[1].protocol_state)

In [ ]:
print(citizen_uni_record_id)

In [ ]:
citizen_uni_cred = await accept_credential(citizen_client_controller, citizen_uni_record_id)
print(citizen_uni_cred)
time.sleep(2)

In [ ]:
time.sleep(3)
uni_cred_record = issue_credential_record(university_client_controller, uni_cred_offer.record_id)

uni_credentials, citizen__uni_credential = check_credential_state(university_client_controller, uni_cred_offer.record_id, citizen_client_controller, citizen_uni_record_id)


# Employer Flow

## Make a Connection between Employer and Citizen

In [ ]:
emp_conn, citizen_emp_conn = await auto_connection(employer_client_controller, citizen_client_controller, "Connection with Decentralized")

## Citizen Present Education Proof to Employe

In [ ]:
emp_proof_request = initiate_proof_request(employer_client_controller, emp_conn.connection_id)
print(emp_proof_request.presentation_id)

In [ ]:
government_proofs,citizen_proofs,university_proofs, employer_proofs, dapp_store_proofs = get_all_presentation_requests()
print('Employer proofs', employer_proofs)
print('\nCitizen proofs', citizen_proofs[1])

In [ ]:
print(citizen_proofs[1].presentation_id)
print(employer_proofs[0].presentation_id)

In [ ]:
citizen_emp_proof_request = accept_proof(citizen_client_controller, citizen_uni_record_id, citizen_proofs[1].presentation_id)

In [ ]:
check_presentation_record_status(university_client_controller, employer_proofs[0].presentation_id, citizen_client_controller, citizen_proofs[1].presentation_id)


## JWT Verify

### Decode JWT in Credential

In [ ]:
citizen_proofs = get_client_presentation_requests(citizen_client_controller)

In [ ]:
# print(citizen_proofs)

In [ ]:
# print(citizen_proofs[1].data[0])

In [ ]:
token = citizen_proofs[1].data[0]

try:
    jwt_edu_decoded_presentation = jwt.decode(token, options={"verify_signature": False})
    print(json.dumps(jwt_edu_decoded_presentation, indent=2))
except (jwt.ExpiredSignatureError, jwt.InvalidAudienceError) as e:
    print("[ERROR]", e)

In [ ]:
token = jwt_edu_decoded_presentation['vp']['verifiableCredential'][0]

try:
    jwt_edu_decoded_vc = jwt.decode(token, options={"verify_signature": False})
    print(json.dumps(jwt_edu_decoded_vc, indent=2))
except (jwt.ExpiredSignatureError, jwt.InvalidAudienceError) as e:
    print("[ERROR]", e)

In [ ]:
jwt_nbf = jwt_decoded_id_token['nbf']
jwt_exp = jwt_decoded_id_token['exp']
print('NBF epoch: {} EXP epoch: {}'.format(jwt_nbf,jwt_exp))

human_jwt_nbf = datetime.datetime.fromtimestamp(jwt_nbf)
human_jwt_exp = datetime.datetime.fromtimestamp(jwt_exp)
print('NBF: {} EXP: {}'.format(human_jwt_nbf,human_jwt_exp))

### Resolve DID

In [ ]:
resolved_did: Response[DIDResponse] = get_did.sync(client=government_client_controller, did_ref=jwt_edu_decoded_vc['iss'])
print(resolved_did.to_dict())

In [ ]:
token = jwt_edu_decoded_presentation['vp']['verifiableCredential'][0]

jwk = resolved_did.did.authentication[0].public_key_jwk
pubKey = jwt.algorithms.ECAlgorithm.from_jwk(json.dumps(jwk.to_dict()))
jwt.decode(token , key = pubKey, algorithms=["ES256K"])

In [ ]:
token = token + 'x'
jwt.decode(token , key = pubKey, algorithms=["ES256K"])

## Employer Issue Employment Credential to Citizen

In [ ]:
# citizen_prism_did = create_prism_did(citizen_client_controller)
# print(citizen_prism_did)
citizen_employment_schema = {
    "firstname": 'Wilmien',
    "lastname": 'Du Preez',
    "email": 'wilmien@decentralized.io',
    "role": "Web3 Developer",
    "photo": "data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAQ4AAAC6CAMAAABoQ1NAAAAAjVBMVEX+/v4AAADt7e3////s7Oz9/f36+vr39/fv7+/y8vLp6ena2trl5eXV1dV4eHi3t7fKysrg4OCqqqpERERlZWXGxsaWlpaenp5UVFS/v7+AgIBsbGxOTk7Ozs6kpKSJiYkbGxs0NDQmJiYRERGxsbEiIiJoaGhTU1NeXl4vLy9JSUkYGBg8PDyNjY10dHQyCsxLAAAPMElEQVR4nO1da1/qPAzf1m1tuQoqIhxUvB31qN//4z29DNjWy9Jule08J28kPyGE/9o0SdM0QhEjhBNORDJUMFQyRDBYMumJQZlg0lwwuWQydBKWDlAyiSIpOrWKTsuiUyw+cxAtmINowUhhB6WHJDnxggNZRaMWSp9Z8j84/sFhgwMJoiknUmaoZEiZEa+ZaE6ZZHLB5JLJBIMlM0DJSYQpJyyJOjC4iXER1hfJUVKAF4l1qAAvEutQAZ4cS/KxyIFZPJYSI0bp6bEIJh2c5AhFSS8mbU8kR//g+AeHIxwVH1YVTSuiOZNWRFOz0jXJ3L4TQQh1K9lT50hqgzNOhWq5ymDJZCWGSoaqjBSW2SVTvg6yDyymY0YLLFb8TiS30flsfgfe3XxfvsRHery8ul9O8zN7NHLG/OykxShZvsVaelrfjFHxnf8PJ53S+ZUei8M42VxgSv4fcBA631vBEPR+l9LkfwAHnV42gyHofiQWm786hMP3QDA43Z3DlIoAhuBcEFEZ+RpXGBkbSYaqjPx8pgjDkw8HNOJ4f42BkrvSOfd2w1KtS1NmqqM0Tei1ExicZiPSLLlLnX/OScdzZzSYTd3l53fSQ8BBnz3QYPT8V8JBb/3QiOMVX2HOAYdtG6Kl7UA7XzTieMPwMEruWOeoYljLTMUWNzGaz1cY+uAPB8PDIrlbnX/E70jougUabIGhf1UIR1et0IjjG/oXOelk2hKNOJ4jreQhwkEoNE6x0EK1jgOFgy7boxG/89WiNyEc9jelSdoBGmK5dTSlPjpHRS2EwCsTrw+JBskU+/2SKR5LYn0sh8KKpGDa2tGCbmldcvc6d+CVVlwadZTmuBs04nhEOvNKTTqHd9LRV1dwfOPhxyyEdIVGHO/I4OGgNw6/d76x/vuD/gAcfn430DtOyDscjRlCC8OGQwFX3ipWAOgc4aCUX8DReGBPCKHbP+Z37POw2mIcNIRjwmdwOC6KYf1tGR7Ub5DCq38i98UCPmkRfgKjMStmLxsgxvdcUt8FrhdOuuWX1ekhL+DgX/Npehf3xQYMh9UyVmh3RIMPENO25RqfAQ7VabNk9C1wIApGY1NCg+Nh2p+aErCj6aEzg0OUNRS1EJIpyh8kU9RSSCY5MSxY4K9ZsMCZXDJZSVgSZfC58oIqcDA8DJHOFxaSA+lMonBrOHuxhcIxraLBAdHj8RiF9TsSR3PgMGlJ+qL9SSrdK2iwz+tdVLYcD9RJJ9DdBJHbUeDQ2+HZYOEAZ4zVqSK/51Hz3ic2LQYKB3A74UuLBpOgTTlfBIUjpCmFobFHBjj05nQT1JQ6lT84bX9lwP2EsQkNZj726tsfsW5jrSOdo7IX47LtCtgche3Zr4xoMFG6kpAdCaZzUCcdFM1+GqeKGB6aeqEVHWbMohnpKi0saDBZmnzJepBwJKAUum2qiOHxS/1MEhAO33moL8oqzUMytqBw2Ka0ThXxlZoSql0wnZNwKwu2WNJfh/9NGtBgSqufvkPhVpZwa7glLb4onvlNExoMD9Ugz+gQQzjNrC/ou4j8L5umChejBj6POJTOIZ108+CgxYLROFW4GI2cCRkcHPVM2Ot8MRpdbOQUkY/8DoAGE6RGPtcDhKOysGxHqAhk1vE7+8P/+RswVbggdR/vjgaDI5gpLScGN8dfjqKPaxShBXCqRNq4dkODmdKGo2KVs3IVJtMcQisJw+VotDQtEF/0EQt2lzA02ONT4HhDYXQmpOyGdXpEkZY308oZjcKwrGFThdtSZevqQRQiBzhWGc5Jx2ULuKn9dOZcjYBosDerlXZcg2HFLLS8df9dhwM/g9HQOWLjocFRLXO5Un48HI1IswX1HAwOJ8MLX4bIoqz+L4dfr8Khpgi/gugs+ncEosry2Bi4WuFQ61LXKJDWwap/KnmbR91GChgOdWvzkSbDCuGqj/QP7hYOHrUMykmvTXjSMRzjocFRLWkCOuRgOK6HBkc1BW7eSwHAockPPtPAcEizJEUXZkmKLsySFF1ijmYpEQ6xNEsnYdX91Ys2cGiyjCsaQmdR3Heywk7V/5HGpEcHYcmkqv68DRyaSt37ADoLJnIYS3APr37ACRq+auHQFEYxr39ITjqtzfeG7RQ7HJrdvPXA4Kg51vWQ1gkOTdnLflhw4FqCc9sGDk2V6UvJOnYKh5MPCw2HqgEcow9/ODTpH1kDFK5/B7SdYa4wxxPLgjl8RFkM3v1jOH1x6gR3rbNgumzBGB0fS6aUDFJ/OLR7vWPStc6CCeCVJlhz4AueC1Tg0NbqTklYr7RDOHSHWHb+cGjPS+0GAwfV1YT5u6VIW7r8k3AU89Ak2lJYwaeKtsUPbANSC4e2dPmiDoe/zmUmOnQgzPP81MFQYQ7tDPMTQyVDqwzF+tpabz9MV+DB4aDd6XwEIO+4BSNCpjZpr95w6IuIdn3v38HgJhfGOmPv5LEmccxpGsgrbQ2HzBFQTG43lUOONV/SN1tqKMcc9xEOKpuwkvHzrKlHh2960FCOueghHOn9+u3tEtbl6NYPDmQoxwwFRxtTmpAluOOTZ8bD1GFrFCqEc1pos8qilWUoW+pOlT+qQeibJxyGMzEL+EJb0TlrWGjbumGYzOvLycul5uD4g99GHPqtReNpFMgN08EhFAE7vJSOV6VthM+tvoLSK4gzOGHxS+iNhRaiCaWYXtxttm9Xm7ubreGcue74XzMchkruzx7DkU4nwlJlo/n9Xq8+J0CJsYLGSHccjtFVMDgKwxt5p1ISwsfDn+bmA867C/rDcJw2Bzi6Tv90kBzMoV0p3PBAluNSd3nnCU2ZHOwihAN37XWoCLOcw2d03V3Y2XkI59ClA44Hym3dHUZ93mexndWoETQrZjSigv5EvYZjYlG9RpbxUcoBMN98bcGYl971Fw6X4WG2p4gc6vgRuon3S/N5GFHFfM4QrqmdIbxvidn/QHQW739vt9urX03dtXZd6KwtaEgK8OS7JXjFuyV4xbulS1NiKqUj2KVD670hM8aWKKBNHpEOdNaWuxzggIwlm4fn0vj7zZQpRIZUYI0ucTc6KzOru30WNAb3j9uzkMOIBwW0QFmFO97TFRxs6VvuYXD8pg989mvRuGCGYbR8ta2ysh9S3+HgJmn6vHldX/5q6NvxC+Wvsjl+HQw8i7diSkd0sTPeyfAZ8qSkiw/btAwVZ3OThtGBuI34WFQBYdySp1SlkrbV6i7gOdqmonUPyhrsIc8TosVlPJug0xk5hJ9f4vvS9oMp8xPHabhOjJ1X/zCB2d4Oh9igRGj+J/59W7TcRNNvfhtLebygzPDxLe1aZ3Q6wBE5TK0mD08qEFlaSgpaHebD9W82VG7mt8+bp/h9SeuTx7DGHNqZ9NdJl6LT1d3y+aY5+XEI4/hzXIrugperMVJMq65EjtEbHUgLRuiNNKUOJnLUykFVJ83xajk4graN14iu5N2TEqMTLQyBFI3vQHDUthiMLqq2LeWWdtCCsaRzDY6mGyiNR3R1d1vaElinHwRNeehs0NOEFl0Xu9K5eqzYxfA2r+EUgAd0t1ZzfpbbHcfF4ryHvwzFP+XnCyxtQPVSXU5X4ny1izk4c6kcvra0suYE3bvWVQy+y82EAcGR0JF9qYUWeui6w4zdF4tzw5GQbGzJ68XXwLmiKey4zYLDEcAsMWFT864AcG9ScwLu4zZ197t70YIRZztTM2vgOqv3Oj5WO4plLUa/WjDaXJqE0nS3MhQGfYLgMAZw8dPV3ah4xl3q3L2TLkWzETeZf1tSfKCF1n7N4Of9BeVXxnelcyg42EMbPTf0igedqW32b9erixQzZ7SvcPCr7cc3zZeegfYlTdF9hR62yzEWIc/ZWzDWW6ESnN7O7BnfgiBlQFqXVEufXzzJ2Fn71m5Wljy7Bl+nAKkh1B9iMQkco361YKSLV7j2T4BON9rwzUwzkvUnhCOO9wI21xCihly8Qnspsw9OOiHgqxQkNfcn0DVlaCCRYusBHGRki1B01OymI4e5V8ajB3CMnO8TrXcRU9HwuWnwgSl0/hAOu44NgJsOr70r02XULuwUptT36pdMXv2CHep+jtRUnu5STFSiGYbprL2uRr6t+H7fdoZ+t9s3uOkIOdytV6ZnDNE5nJNOHGrkStTgpsNd0ho9kfPGLBh+s1eZGtx0J5e0QpuzwkG9TB6vaLDD4X/3tbzy+jyHvxLvO7utcKCRNxq81bhdZ/vS2aoFY+45OOI4teEBvMhET7ftaj9aVf/4WY64wU03XhkIoT1vznieEM5vWeFka1BgrvoBEe83dx4n3f+Wapub7ueSHumdkvPAoXa0AdPeBofDFbY6eqZngYO61BnXyeymowh8UllPlwA4VLf7AEfRrke8zsTrpIBDMkJ0hCVTiJYfaSoBs5Hlti/bLTcguiZGnW39ioqvL1lhp+p/nyD8SGY33XSsGk6vbicWujn85VSTr5DposDIVCDnQhNq0Dmgk67vUASltQmONi7pgb7wj8OBcvil3Rp6MNbHuWWhtfQ0Ij8OB/RyVQOZ3HRD9aQble9v+aEQDlIjaCFDW7WWLmlBnyTxC+GOLRiNm1SGHSvtXbEOZHDT2xnoI+0ywy5bmdHt0pXdMMCtmXIypQnR9NFzopkBjjbOzIm2CKk6QzahPb1SSP2olfTZdN01X16U1uEI66TTlnPFUPTS1kAfafWjcLSeK4abRlsEyVV6ilB3cFRshwoH7QIOrZsOu7MVQtfIz3YIu0oKw0oEk6sMlsyxZA+3XgF0bnqLhFKNrnCm6AwpYPH2O9rOcl3RC4IdAGmmL0p+uH+HuRENjB41RS+WWzmdaI7Jj++zdO+mOxe56Olh8vMhHBNNJ/oWZ0DaIYU6cUnf8Jl24Uj96gAnEncOY9GjiG+kp+lo5FHkotASoTPtwjFvbNwyz9sxva9wk84NprTpqFhePiqmHJyL0GjMaTESNG5gJuL1hL+cTieCFlNBBSPfJj+ycJecY9qsc+3gHC0B4N+C8VQbJkQX40qKLsaVhLPMyCdRHN2T/8jLDFaFOUmG6qxm0sN10feZtD2RHPIu639w/AVwFIY3irpuZ5gOUHJUJMokUQcGNzEuwvoiOdQFrKljYUU/JIe7rbgv5qAXV139g+PvhkNX/nAQHVlFRw1K91jysZ1JYkU6KYtOypsySdmjT4oVTrx2KqzoiWTyH0OmNgTTsTIxAAAAAElFTkSuQmCC",
  }

employer_cred = create_credential(citizen_employment_schema, "schema:12345:employment", emp_conn.their_did)
employer_cred_offer = send_credential_offer(employer_client_controller, employer_cred)
time.sleep(2)

In [ ]:
citizen_emp_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=citizen_client_controller)
citizen_emp_record_id = citizen_emp_creds.items[1].record_id
print(citizen_emp_creds.items[1].protocol_state)

In [ ]:
# while (citizen_creds.items[0].protocol_state == 'RequestSent' or citizen_creds.items[0].protocol_state != 'OfferReceived'):
#     citizen_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=citizen_client_controller)
#     print(citizen_creds)

In [ ]:
citizen_emp_cred = await accept_credential(citizen_client_controller, citizen_emp_record_id)
print(citizen_emp_cred)
time.sleep(2)

In [ ]:
time.sleep(2)
emp_cred_record = issue_credential_record(employer_client_controller, employer_cred_offer.record_id)

emp_credentials, citizen_emp_credential = check_credential_state(employer_client_controller, employer_cred_offer.record_id, citizen_client_controller, citizen_emp_record_id)


# Dapp Store Flow

## Make connection between Holder and Dapp Store

In [ ]:
dapp_conn, citizen_dapp_conn = await auto_connection(dapp_store_client_controller, citizen_client_controller, "Connection with Dapp Store")

## Citizen Present Employment Proof to Dapp Store Code

In [ ]:
dapp_proof_request = initiate_proof_request(dapp_store_client_controller, dapp_conn.connection_id)
print(dapp_proof_request.presentation_id)

In [ ]:
government_proofs,citizen_proofs,university_proofs, employer_proofs, dapp_store_proofs = get_all_presentation_requests()
print('Dapp proofs', dapp_store_proofs)
print('\nCitizen proofs', citizen_proofs[2])

In [ ]:
print(citizen_proofs[1].presentation_id)
print(employer_proofs[0].presentation_id)

In [ ]:
citizen_emp_proof_request = accept_proof(citizen_client_controller, citizen_uni_record_id, citizen_proofs[1].presentation_id)

In [ ]:
check_presentation_record_status(university_client_controller, employer_proofs[0].presentation_id, citizen_client_controller, citizen_proofs[1].presentation_id)


## JWT Verify

### Decode JWT in Credential

In [ ]:
citizen_proofs = get_client_presentation_requests(citizen_client_controller)

In [ ]:
# print(citizen_proofs)

In [ ]:
# print(citizen_proofs[1].data[0])

In [ ]:
token = citizen_proofs[1].data[0]

try:
    jwt_edu_decoded_presentation = jwt.decode(token, options={"verify_signature": False})
    print(json.dumps(jwt_edu_decoded_presentation, indent=2))
except (jwt.ExpiredSignatureError, jwt.InvalidAudienceError) as e:
    print("[ERROR]", e)

In [ ]:
token = jwt_edu_decoded_presentation['vp']['verifiableCredential'][0]

try:
    jwt_edu_decoded_vc = jwt.decode(token, options={"verify_signature": False})
    print(json.dumps(jwt_edu_decoded_vc, indent=2))
except (jwt.ExpiredSignatureError, jwt.InvalidAudienceError) as e:
    print("[ERROR]", e)

In [ ]:
jwt_nbf = jwt_decoded_id_token['nbf']
jwt_exp = jwt_decoded_id_token['exp']
print('NBF epoch: {} EXP epoch: {}'.format(jwt_nbf,jwt_exp))

human_jwt_nbf = datetime.datetime.fromtimestamp(jwt_nbf)
human_jwt_exp = datetime.datetime.fromtimestamp(jwt_exp)
print('NBF: {} EXP: {}'.format(human_jwt_nbf,human_jwt_exp))

### Resolve DID

In [ ]:
resolved_did: Response[DIDResponse] = get_did.sync(client=government_client_controller, did_ref=jwt_edu_decoded_vc['iss'])
print(resolved_did.to_dict())

In [ ]:
token = jwt_edu_decoded_presentation['vp']['verifiableCredential'][0]

jwk = resolved_did.did.authentication[0].public_key_jwk
pubKey = jwt.algorithms.ECAlgorithm.from_jwk(json.dumps(jwk.to_dict()))
jwt.decode(token , key = pubKey, algorithms=["ES256K"])

In [ ]:
token = token + 'x'
jwt.decode(token , key = pubKey, algorithms=["ES256K"])

## Dapp Store Issue Verified Dapp Credential to Citizen

In [ ]:
# citizen_prism_did = create_prism_did(citizen_client_controller)
# print(citizen_prism_did)
citizen_dapp_schema = {
    "dapp_name": 'Wilmien',
    "category": 'Du Preez',
    "website": '11/11/1999',
    "description": "Pretoria",
    "dapp_logo": "...",
    "issuedate": "21/12/2022",
    "expiry_date": "23/12/2022"
  }

dapp_cred = create_credential(citizen_dapp_schema, "schema:12345:dapp", dapp_conn.their_did)
dapp_cred_offer = send_credential_offer(dapp_store_client_controller, dapp_cred)
time.sleep(2)

In [ ]:
citizen__dapp_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=citizen_client_controller)
citizen_dapp_record_id = citizen__dapp_creds.items[3].record_id
print(citizen__dapp_creds.items[3].protocol_state)

# citizen_uni_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=citizen_client_controller)
# citizen_uni_record_id = citizen_uni_creds.items[1].record_id
# print(citizen_uni_creds.items[1].protocol_state)


In [ ]:
# while (citizen_creds.items[0].protocol_state == 'RequestSent' or citizen_creds.items[0].protocol_state != 'OfferReceived'):
#     citizen_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=citizen_client_controller)
#     print(citizen_creds)

In [ ]:
citizen__dapp_cred = await accept_credential(citizen_client_controller, citizen_dapp_record_id)
print(citizen__dapp_cred)
time.sleep(2)

In [ ]:
dapp_cred_record = issue_credential_record(dapp_store_client_controller, dapp_cred_offer.record_id)

dapp_credentials, citizen__dapp_credential = await check_credential_state(dapp_store_client_controller, dapp_cred_offer.record_id, citizen_client_controller, citizen_dapp_record_id)
